In [1]:
!pip install scikit-learn 

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


## La previa

In [6]:
captures = ["10","11","12","15","15-2","16","16-2","16-3","17","18","18-2","19","15-3"]
        
# para poder descargar los archivos con las features y los ncol de cada captura
features_links = ["ktAgJS22kBXcMsT", 
                    "qfM8pjZgDi9EFfd",
                    "34PWCn9JmdDA9HE",
                    "YtcCmiNEyPiYB4Y",
                    "D2HdaEZYcFz5ryt",
                    "cTY262moFLGjtmn",
                    "TbfPFbSKWqYqR7n",
                    "s2GTjFz8rNxbs4z",
                    "9dZPAENNACreDEK",
                    "bwR2Zrky49JjtgA",
                    "CmYc9JyBsHwzaYD",
                    "TNSkGJcq2CPoFtM",
                    "XwZFrQYzMLNJxAY"
]

for i in range(len(captures)):
    x_tmp = pd.read_csv(f'https://nube.ingenieria.uncuyo.edu.ar/s/{features_links[i]}/download', sep=",", header=0)
    x = x_tmp.iloc[:,0:5]
    y_tmp = []
    for j in range(x_tmp.shape[0]):
        if (x.iloc[j,4] > 2):
            y_tmp.append("infected")
        else:
            y_tmp.append("normal")
    y = pd.DataFrame(y_tmp,columns=["label"])
    xy = pd.concat([x, y], axis=1)
    xy.to_csv(f'/mnt/RF/capture201108{captures[i]}_paraRF.csv', sep=",", index=False)

In [14]:
x_tmp = pd.read_csv("https://nube.ingenieria.uncuyo.edu.ar/s/ktAgJS22kBXcMsT/download", sep=",", header=0)

In [15]:
x = x_tmp.iloc[:,0:5]

In [22]:
x.iloc[0,3]

0

In [ ]:
# En R, junté todas las capturas (para después hacer el diccionario node_idx)
# En R, junté las capturas que me interesan para entrenar


# df_features <- readr::read_csv("capture20110810_paraRF.csv", show_col_types = FALSE)
# aux = c("20110811", "20110812", "20110815", "20110815-2", "20110816", "20110816-2", "20110816-3", "20110817", "20110818", "20110818-2", "20110819", "20110815-3")
# i = 2
# for (date in aux) {
#     cap_features <- readr::read_csv(paste("capture", date, "_paraRF.csv", sep = ""), show_col_types = FALSE)
#     df_features <- dplyr::bind_rows(df_features, cap_features)
#     i <- i + 1
# }

# readr::write_csv(df_features, "all_captures_FEATURES.pkts.SINnorm.csv")
        
# ###################

# df_features <- readr::read_csv("capture20110810_paraRF.csv", show_col_types = FALSE)

# # Captures in order from 2nd to 13th
# aux = c("20110811", "20110812", "20110815", "20110815-2", "20110816", "20110816-2", "20110816-3", "20110817", "20110818", "20110818-2", "20110819", "20110815-3")

# for (date in aux) {
#     if ( (date != "20110816-3") & (date != "20110817") ) {
#         cap_features <- readr::read_csv(paste("capture", date, "_paraRF.csv", sep = ""), show_col_types = FALSE)
#         df_features <- dplyr::bind_rows(df_features, cap_features)
#     }
# }

# readr::write_csv(df_features, "all_captures_except_8y9_FEATURES.pkts.SINnorm.csv")


## Ahora sí

In [2]:
x_tmp = pd.read_csv("/mnt/RF/all_captures_FEATURES.pkts.SINnorm.csv", sep=",", header=0)

In [3]:
x = pd.read_csv("/mnt/RF/all_captures_except_8y9_FEATURES.pkts.SINnorm.csv", sep=",", header=0)

In [13]:
x.head

<bound method NDFrame.head of                                  node      ID      OD      IDW      ODW  \
0                           1-0.0.0.0       0       1        0     2239   
1                   1-255.255.255.255       6       0     2907        0   
2                     1-1.108.169.143       1       1        6        6   
3                     1-147.32.84.229  525631  540374  4796296  4936679   
4                         1-1.11.9.68       1       1        2        2   
...                               ...     ...     ...      ...      ...   
2492758     13-fe80::5054:ff:fe89:354       0       1        0        4   
2492759                  13-ff02::1:3       4       0       48        0   
2492760    13-fe80::5054:ff:feef:6697       0       1        0        4   
2492761  13-fe80::d036:46ef:ad35:fc09       0       1        0        6   
2492762  13-fe80::f180:4f5c:7d4a:bbd7       0       1        0       34   

            label  
0        infected  
1          normal  
2        

In [4]:
node_idx = {name: idx for idx, name in enumerate(sorted(x_tmp["node"].unique()))}
class_idx = {name: idx for idx, name in enumerate(sorted(x_tmp["label"].unique()))} # 0=infected, 1=normal

In [5]:
x["node"] = x["node"].apply(lambda name: node_idx[name])
x["label"] = x["label"].apply(lambda value: class_idx[value])

In [17]:
x.head

<bound method NDFrame.head of             node      ID      OD      IDW      ODW  label
0              0       0       1        0     2239      0
1         252661       6       0     2907        0      1
2              1       1       1        6        6      0
3          97219  525631  540374  4796296  4936679      0
4              2       1       1        2        2      1
...          ...     ...     ...      ...      ...    ...
2492758  1251096       0       1        0        4      0
2492759  1251100       4       0       48        0      1
2492760  1251097       0       1        0        4      0
2492761  1251098       0       1        0        6      0
2492762  1251099       0       1        0       34      0

[2492763 rows x 6 columns]>

In [6]:
X = x.iloc[:,0:5]
X.head

<bound method NDFrame.head of             node      ID      OD      IDW      ODW
0              0       0       1        0     2239
1         252661       6       0     2907        0
2              1       1       1        6        6
3          97219  525631  540374  4796296  4936679
4              2       1       1        2        2
...          ...     ...     ...      ...      ...
2492758  1251096       0       1        0        4
2492759  1251100       4       0       48        0
2492760  1251097       0       1        0        4
2492761  1251098       0       1        0        6
2492762  1251099       0       1        0       34

[2492763 rows x 5 columns]>

In [7]:
y = x.iloc[:,5:6]
y.head

<bound method NDFrame.head of          label
0            0
1            1
2            0
3            0
4            1
...        ...
2492758      0
2492759      1
2492760      0
2492761      0
2492762      0

[2492763 rows x 1 columns]>

In [8]:
suma = 0
for i in range(len(X)):
    if (y.iloc[i,0] == 0): # 0 == "infected"
        suma += 1

print("infected:",suma," - normal:", len(X)-suma, " -- total:",len(X))

infected: 1115617  - normal: 1377146  -- total: 2492763


In [9]:
print("infected/total = ",suma/len(X))

infected/total =  0.44754234558199074


In [10]:
x_test_Tati = pd.read_csv("/mnt/RF/capture20110816-3_paraRF.csv", sep=",", header=0)

x_test_Tati["node"] = x_test_Tati["node"].apply(lambda name: node_idx[name])
x_test_Tati["label"] = x_test_Tati["label"].apply(lambda value: class_idx[value])

X_test_Tati = x_test_Tati.iloc[:,0:5]
y_test_Tati = x_test_Tati.iloc[:,5:6]

In [12]:
y_test_Tati

,label
0,0
1,1
2,1
3,0
4,1
...,...
381445,0
381446,1
381447,0
381448,0


In [13]:
# Split the data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_clf.fit(X, y)

# Make predictions on the test set
y_pred = rf_clf.predict(X_test_Tati)

# Calculate the accuracy
accuracy = accuracy_score(y_test_Tati, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print the confusion matrix
conf_matrix = confusion_matrix(y_test_Tati, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print the classification report
class_report = classification_report(y_test_Tati, y_pred)
print("Classification Report:\n", class_report)

<ipython-input-13-3420f943dcb0>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X, y)


Accuracy: 1.00
Confusion Matrix:
 [[189679      0]
 [     0 191771]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    189679
           1       1.00      1.00      1.00    191771

    accuracy                           1.00    381450
   macro avg       1.00      1.00      1.00    381450
weighted avg       1.00      1.00      1.00    381450



**----------------------------------------------------------------------------------------------------------------**

In [ ]:
# En R, junté todas las capturas (para después hacer el diccionario node_idx)
# En R, junté las capturas que me interesan para entrenar


# df_features <- readr::read_csv("capture20110810_features_prefix.csv", show_col_types = FALSE)
# aux = c("20110811", "20110812", "20110815", "20110815-2", "20110816", "20110816-2", "20110816-3", "20110817", "20110818", "20110818-2", "20110819", "20110815-3")

# for (date in aux) {
#     cap_features <- readr::read_csv(paste("capture", date, "_features_prefix.csv", sep = ""), show_col_types = FALSE)
#     df_features <- dplyr::bind_rows(df_features, cap_features)
# }

# readr::write_csv(df_features, "all_captures_FEATURES_originales.pkts.SINnorm.csv")
        
# # ###################

# df_features <- readr::read_csv("capture20110810_features_prefix.csv", show_col_types = FALSE)

# # Captures in order from 2nd to 13th
# aux = c("20110811", "20110812", "20110815", "20110815-2", "20110816", "20110816-2", "20110816-3", "20110817", "20110818", "20110818-2", "20110819", "20110815-3")

# for (date in aux) {
#     if ( (date != "20110816-3") & (date != "20110817") ) {
#         cap_features <- readr::read_csv(paste("capture", date, "_features_prefix.csv", sep = ""), show_col_types = FALSE)
#         df_features <- dplyr::bind_rows(df_features, cap_features)
#     }
# }

# readr::write_csv(df_features, "all_captures_except_8y9_FEATURES_originales.pkts.SINnorm.csv")


In [23]:
x_tmp_orig = pd.read_csv("/mnt/RF/all_captures_FEATURES_originales.pkts.SINnorm.csv", sep=",", header=0)
x_orig = pd.read_csv("/mnt/RF/all_captures_except_8y9_FEATURES_originales.pkts.SINnorm.csv", sep=",", header=0)

In [24]:
node_idx = {name: idx for idx, name in enumerate(sorted(x_tmp_orig["node"].unique()))}
class_idx = {name: idx for idx, name in enumerate(sorted(x_tmp_orig["label"].unique()))} # 0=infected, 1=normal

In [25]:
x_orig["node"] = x_orig["node"].apply(lambda name: node_idx[name])
x_orig["label"] = x_orig["label"].apply(lambda value: class_idx[value])

In [26]:
X_orig = x_orig.iloc[:,0:5]
X_orig.head

<bound method NDFrame.head of             node      ID      OD      IDW      ODW
0              0       0       1        0     2239
1         252661       6       0     2907        0
2              1       1       1        6        6
3          97219  525631  540374  4796296  4936679
4              2       1       1        2        2
...          ...     ...     ...      ...      ...
2492758  1251096       0       1        0        4
2492759  1251100       4       0       48        0
2492760  1251097       0       1        0        4
2492761  1251098       0       1        0        6
2492762  1251099       0       1        0       34

[2492763 rows x 5 columns]>

In [27]:
y_orig = x_orig.iloc[:,5:6]
y_orig.head

<bound method NDFrame.head of          label
0            1
1            1
2            1
3            1
4            1
...        ...
2492758      1
2492759      1
2492760      1
2492761      1
2492762      1

[2492763 rows x 1 columns]>

In [28]:
suma = 0
for i in range(len(X_orig)):
    if (y_orig.iloc[i,0] == 0): # 0 == "infected"
        suma += 1

print("infected:",suma," - normal:", len(X_orig)-suma, " -- total:",len(X_orig))

infected: 24  - normal: 2492739  -- total: 2492763


In [29]:
x_test_Tati_orig = pd.read_csv("/mnt/features-prefix/capture20110816-3_features_prefix.csv", sep=",", header=0)

x_test_Tati_orig["node"] = x_test_Tati_orig["node"].apply(lambda name: node_idx[name])
x_test_Tati_orig["label"] = x_test_Tati_orig["label"].apply(lambda value: class_idx[value])

X_test_Tati_orig = x_test_Tati_orig.iloc[:,0:5]
y_test_Tati_orig = x_test_Tati_orig.iloc[:,5:6]

In [30]:
# Split the data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest classifier
rf_clf_orig = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_clf_orig.fit(X_orig, y_orig)

# Make predictions on the test set
y_pred_orig = rf_clf_orig.predict(X_test_Tati_orig)

# Calculate the accuracy
accuracy_orig = accuracy_score(y_test_Tati_orig, y_pred_orig)
print(f"Accuracy: {accuracy_orig:.2f}")

# Print the confusion matrix
conf_matrix_orig = confusion_matrix(y_test_Tati_orig, y_pred_orig)
print("Confusion Matrix:\n", conf_matrix_orig)

# Print the classification report
class_report_orig = classification_report(y_test_Tati_orig, y_pred_orig)
print("Classification Report:\n", class_report_orig)

<ipython-input-30-600c8609551c>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf_orig.fit(X_orig, y_orig)


Accuracy: 1.00
Confusion Matrix:
 [[     0      1]
 [     0 381449]]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00    381449

    accuracy                           1.00    381450
   macro avg       0.50      0.50      0.50    381450
weighted avg       1.00      1.00      1.00    381450



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
